In [1]:
import os 
import csv 
from tqdm import tqdm
# from config import Config

def createCsv(data_dir, write_file):
    toCsv = []
    for root, files, name in tqdm(os.walk(data_dir)):      
        if root != data_dir:
            for x in name:
                if '.png' in x:
                    # index = root[root.index(os.sep)+1:]
                    index = root.split(os.sep)[-1]
                    # print(f"{x}, path={os.path.join(root, x)}, index={index}")
                    toCsv.append([os.path.join(root, x), int(index)])
                    
                    
    toCsv.sort(key=lambda x: x[1])
    toCsv = [x for x in toCsv]
    # print(toCsv)

    with open(os.path.join(write_file), "w", newline='') as f:
        writer = csv.writer(f)
        for x in range(len(toCsv)):
            writer.writerow(toCsv[x])
            

DATA_DIR = "/dcs/large/u2009169/seal-script-images/"
TARGET_FILE = "./train.csv"

createCsv(DATA_DIR, TARGET_FILE)

0it [00:00, ?it/s]

1077it [00:12, 85.82it/s]


In [1]:
from config import Config_2 as Config
import train

model_types = ['resnet_50', 'mobilenet_v3_large']
model, criterion, optimiser = train.init_model(1076, model_types[1], Config.LEARNING_RATE, Config.ADAM_BETA)

train.load_model(model, optimiser, "/dcs/large/seal-script-project-checkpoints/mobilenetv3large/2024-04-03/CK-59.pt")

==========Logger started===========
False
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /dcs/20/u2002183/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth


False


100%|██████████| 21.1M/21.1M [00:00<00:00, 101MB/s] 
Using device: cpu
Pretrained: 0.0001
MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0):

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [6]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
import torch 
import torch.onnx as onnx 

x = torch.rand(1, 3, 224, 224).to("cuda")
torch_out = onnx.export(model, x, "build/seals-resnet50.onnx", export_params=True)